## Установка библиотек

In [ ]:
%pip install --quiet --upgrade mediapy python-telegram-bot telegram redis telebot
%pip install --quiet --upgrade diffusers accelerate mediapy triton scipy ftfy transformers
%pip install --quiet spacy==3.5.0
%pip install --quiet torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1
# Пакет xformers обязателен для создания нескольких изображений размером 768x768
%pip install --quiet xformers==0.0.16rc425

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 78.8 MB/s eta 0:00:00


## Импорт модулей

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import requests
import redis
import json
import io

## Генерация и отправка изображений домов в Telegram

In [ ]:
# Настройки модели Diffusion
device = "cuda"
model_id = "stabilityai/stable-diffusion-2-1"
model_revision = "fp16"
image_length = 768

# Настройки подключения к Redis
redis_host = 'redis-13865.c1.asia-northeast1-1.gce.cloud.redislabs.com'
redis_port = 13865
redis_password = 'AjuhoBzf8IF3TmxG1SE3yFoMWh2q19Kt'

# Создание экземпляра клиента Redis
r = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

bot_token = "6266329963:AAFAo5KOFAjRwUNFUGaX-WchaRXA82CA7iI"
url = f"https://api.telegram.org/bot{bot_token}/sendPhoto"


def main():
    def generate_and_send_image(house_data):
        user_id = house_data['id']

        # Генерация изображения на основе параметров дома
        prompt = "Generate an image of a house with the following specifications:"
        for key, value in house_data.items():
            if key != "id" and key != "image":
                prompt += f"\n{key.capitalize()}: {value}"

        # Создание и настройка конвейера
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            revision=model_revision,
        )
        pipe = pipe.to(device)
        pipe.enable_xformers_memory_efficient_attention()

        # Генерация изображений
        images = pipe(
            prompt,
            height=image_length,
            width=image_length,
            num_inference_steps=25,
            guidance_scale=9,
            num_images_per_prompt=1,
            generator=torch.Generator("cuda")
        ).images

        # Преобразование изображения в байты
        byte_stream = io.BytesIO()
        images[0].save(byte_stream, format='JPEG')
        image_bytes = byte_stream.getvalue()

        params = {
            'chat_id': user_id
        }
        files = {
            "photo": ("image.jpeg", image_bytes)
        }
        response = requests.post(url=url, params=params, files=files)

        # Проверка статуса кода ответа
        if response.status_code == 200:
            print(f'Фото успешно отправлено в Telegram для пользователя c ID {user_id}')
        else:
            print('Произошла ошибка при отправке фото в Telegram')
            print(f"Ошибка при отправке фото в Telegram. Код ответа: {response.status_code}")
            print(f"Текст ошибки: {response.text}")

        r.delete(user_id)

    while True:
        keys = r.keys("*")
        for item in keys:
            data = (r.mget(item))[0]
            data = data.decode()
            house_data = json.loads(data)
            generate_and_send_image(house_data)


main()